<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Parcial%202-6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.stats import expon

# --- Parámetros del Problema y Simulación ---
lambda_X = 1.0  # Parámetro de X
lambda_Y = 2.0  # Parámetro de Y
Threshold = 1.5 # Umbral de la suma (c = 1.5)
N = 1000000     # Número de simulaciones

# --- Valor Analítico de Referencia (Para comparación) ---
# P(X + Y < c) = 1 - (lambda_Y / (lambda_Y - lambda_X)) * exp(-lambda_X * c) + (lambda_X / (lambda_Y - lambda_X)) * exp(-lambda_Y * c)
# Asumiendo lambda_Y != lambda_X
P_analytic = 1 - (lambda_Y / (lambda_Y - lambda_X)) * np.exp(-lambda_X * Threshold) + \
             (lambda_X / (lambda_Y - lambda_X)) * np.exp(-lambda_Y * Threshold)

print(f"Valor Analítico P(X + Y < 1.5): {P_analytic:.6f}")
print("-" * 50)


# --- A) Simulación Simple (Monte Carlo Directo) ---
print("--- A) Simulación Simple ---")

# 1. Generar N muestras para X y Y
# La función inversa es F^{-1}(U) = -1/lambda * log(1-U)
U_X = np.random.uniform(0, 1, N)
U_Y = np.random.uniform(0, 1, N)

X_simple = -1/lambda_X * np.log(1 - U_X)
Y_simple = -1/lambda_Y * np.log(1 - U_Y)

# 2. Estimar la probabilidad P(X + Y < 1.5)
Sum_simple = X_simple + Y_simple
indicator_simple = (Sum_simple < Threshold).astype(int)

# Estimador: P_hat = E[I(X + Y < c)]
P_hat_simple = np.mean(indicator_simple)

# 3. Calcular la varianza del estimador
Var_indicator_simple = np.var(indicator_simple, ddof=1)
Var_P_hat_simple = Var_indicator_simple / N

print(f"Estimación P(X + Y < 1.5) (Simple): {P_hat_simple:.6f}")
print(f"Varianza del Estimador (Simple): {Var_P_hat_simple:.10f}")
print("-" * 50)

# --- B) Técnica de Condicionamiento (Muestreo Estratificado) ---
print("--- B) Técnica de Condicionamiento ---")

# Usamos el estimador de condicionamiento: E[I(X + Y < c)] = E[E[I(X + Y < c) | X]]
# E[I(X + Y < c) | X=x] = P(X + Y < c | X=x) = P(Y < c - x | X=x)
# Como X e Y son independientes: P(Y < c - x)

# P(Y < c - x) es la CDF de Y evaluada en (c - x):
# F_Y(c - x) = 1 - exp(-lambda_Y * (c - x)), para c - x > 0 (es decir, x < c)
# Si x >= c, P(Y < c - x) = 0.

# 1. Muestrear N valores de X (Variable de condicionamiento)
# Usamos las mismas muestras X_simple para una comparación justa de varianza
X_cond = X_simple

# 2. Calcular el estimador condicional g_cond(X) = E[I(X + Y < c) | X]
# Inicializar el estimador condicional a 0
g_cond_X = np.zeros(N)

# Encontrar los índices donde x < c (donde la probabilidad no es 0)
idx = X_cond < Threshold

# Calcular la CDF de Y evaluada en (Threshold - X)
# F_Y(c - x) = 1 - exp(-lambda_Y * (c - x))
g_cond_X[idx] = 1 - np.exp(-lambda_Y * (Threshold - X_cond[idx]))

# 3. Estimar la probabilidad P(X + Y < 1.5) con el estimador condicional
# P_hat_cond = E[g_cond(X)] = mean(g_cond_X)
P_hat_cond = np.mean(g_cond_X)

# 4. Calcular la varianza del estimador condicional
# Var[P_hat_cond] = Var[g_cond(X)] / N
Var_g_cond_X = np.var(g_cond_X, ddof=1)
Var_P_hat_cond = Var_g_cond_X / N

# 5. Porcentaje de Reducción de Varianza
Reduccion_cond = (Var_P_hat_simple - Var_P_hat_cond) / Var_P_hat_simple * 100

print(f"Estimación P(X + Y < 1.5) (Condicionamiento): {P_hat_cond:.6f}")
print(f"Varianza del Estimador (Condicionamiento): {Var_P_hat_cond:.10f}")
print(f"Reducción de Varianza: {Reduccion_cond:.2f}%")

Valor Analítico P(X + Y < 1.5): 0.603527
--------------------------------------------------
--- A) Simulación Simple ---
Estimación P(X + Y < 1.5) (Simple): 0.603507
Varianza del Estimador (Simple): 0.0000002393
--------------------------------------------------
--- B) Técnica de Condicionamiento ---
Estimación P(X + Y < 1.5) (Condicionamiento): 0.603769
Varianza del Estimador (Condicionamiento): 0.0000001392
Reducción de Varianza: 41.81%
